# Notebook Playground

This notebook's goal is to get used to using OpenCV to work with basic images. It relies on images stored in the `data` directory.

I've taken photos of my desktop monitor displaying a checkerboard pattern, and hopefully can do some useful camera calibration stuff with OpenCV's functionality.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# My utility functions
from utils import *

# For timing tests
import time

In [ ]:
img_name = "Checkerboard/IMG_0821.jpg"
print(f"Loading image {img_name}")
image = load_image(img_name)
plt.imshow(image)
plt.show()
image = convert_image_c2f(image)
print(image.shape)
print(image[0:5, 0:5, :])

In [ ]:
files = glob_data("checkerboard_video/*")
for f in files:
    img = load_image(f)
    print(f"img.shape {img.shape}")

## Image filters

`cv2.filter2D` is OpenCV's implementation of applying an image filter, such as a sobel filter, to an image. It can be an effective way to find edges in an image, depending on the kernel used.

A good source to learn about kernels is [Wikipedia](https://en.wikipedia.org/wiki/Kernel_(image_processing))

In [ ]:
def edge_filter_pipeline(img_name):
    image = load_image(img_name, grayscale=True)
    image = convert_image_c2f(image)

    # Smooth the image with a gaussian blur to account for noise
    ksize = (0, 0) # zeros cause kernel size to be determined by sigma
    sigma = 10.0
    image_blurred = cv2.GaussianBlur(image, ksize, sigma)
    plt.imshow(image_blurred, cmap='gray')
    plt.show()

    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])
    image_edges = cv2.filter2D(image_blurred, -1, kernel)
    min_val = np.amin(image_edges)
    range_val = np.amax(image_edges - min_val) 
    image_edges = (image_edges - min_val) / range_val
    plt.imshow(image_edges, cmap='gray')
    plt.show()

    threshold = 0.70
    image_thresholded = np.where(image_edges > threshold, 1.0, 0.0)
    plt.imshow(image_thresholded, cmap='gray')
    plt.show()

In [ ]:
for f in files[6:7]:
    edge_filter_pipeline(f)

## Camera calibration using a checkerboard (or chessboard)

OpenCV uses the chessboard pattern to calibrate cameras.

It's all very complex and detailed, but lets give it a go!

In [ ]:
# Constants related to camera checkerboard calibration.

# Checkerboard dimensions - number of internal corners (row, col)
PATTERN_SIZE = (5, 7)
PATTERN_NUM_POINTS = PATTERN_SIZE[0] * PATTERN_SIZE[1]

# Pixel refinement termination criteria - accuracy and number of iterations
CRITERIA = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [ ]:
def get_chessboard_corners(img_name, reverse_image=False, debug=False):
    """
    Tries to find the chessboard in the image given by img_name.
    Flag determines if should plot/debug result
    Returns:
        - ret: boolean success value
        - corners: subpixel coordinates of found corners, or None otherwise
    """
    image = load_image(img_name, grayscale=True)
    if reverse_image:
        image = -image + 255
    
    ret, corners = cv2.findChessboardCorners(image, PATTERN_SIZE)
    
    if not ret:
        if debug:
            print(f"cv2.findChessboardCorners failed for {img_name}")
        return False, None

    corners2 = cv2.cornerSubPix(image, corners, (11, 11), (-1, -1), CRITERIA)
    
    if debug:
        plt.imshow(image,cmap='gray')
        plt.scatter(corners2[:,0,0], corners2[:,0,1], c='y', marker='.')
        plt.show()

    return ret, corners


In [ ]:
horizontal_image_paths = []
image_shape = None
for f in files[:]:
    image = load_image(f)
    if image.shape[0] < image.shape[1]:
        image_shape = image.shape[:2]
        horizontal_image_paths.append(f)
    else:
        print(image.shape)


In [ ]:
# Calibration goal: Run cv2.calibrateCamera to get the intrinsic parameters
# Construct a list of object points representing the internal corners of the checkerboard
objp = np.zeros((PATTERN_NUM_POINTS, 3), np.float32)
objp[:,:2] = np.mgrid[0:PATTERN_SIZE[0],0:PATTERN_SIZE[1]].T.reshape(-1, 2)

# Requires the following parameters:
objpoints = [] # list of objp, representing the 3d coordinates of our corners
imgpoints = [] # List of the found 2d corners in each image plane
image_size = image_shape[::-1]
camera_matrix = None # Initial guess for camera matrix, and output param
distortion_coeffs = None

In [ ]:
ret, corners = get_chessboard_corners(horizontal_image_paths[0], debug=True)
print(f"Got corners? {ret}")
print(f"Corners got: {corners}")

In [ ]:

average_delta_time = 0.0

num_imgs = len(horizontal_image_paths)
for f in horizontal_image_paths:
    img_start_time = time.time()
    ret, corners = get_chessboard_corners(f)
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
    img_end_time = time.time()
    average_delta_time += (img_end_time - img_start_time)/num_imgs
    print(f"Image {f} took {img_end_time - img_start_time} seconds to get corners")

print(f"Average time per image: {average_delta_time} seconds")

In [ ]:
start_time = time.time()

calibration_start_time = time.time()
ret, intrinsic_mat, distortion_mat, rotation_vecs, translation_vecs = \
    cv2.calibrateCamera(objpoints, imgpoints, image_size, camera_matrix, distortion_coeffs)
end_time = time.time()

print(f"Calibration took {end_time - calibration_start_time} seconds")
print(f"Total time: {average_delta_time * num_imgs + end_time - start_time} seconds")


In [ ]:
print(intrinsic_mat)
print(distortion_mat)
numpy_save("numpy/intrinsic_mat.npy", intrinsic_mat)
numpy_save("numpy/distortion_mat.npy", distortion_mat)

In [ ]:
img = load_image(horizontal_image_paths[1])
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(intrinsic_mat, distortion_mat, (w,h), 1, (w,h))

# undistort
dst = cv2.undistort(img, intrinsic_mat, distortion_mat, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
print("Original")
plt.imshow(img)
plt.show()

print("Undistorted")
plt.imshow(dst)
plt.show()